<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initial-data-exploration-and-cleaning" data-toc-modified-id="Initial-data-exploration-and-cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initial data exploration and cleaning</a></span></li><li><span><a href="#Extracting-past-care-units-features" data-toc-modified-id="Extracting-past-care-units-features-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extracting past care units features</a></span></li><li><span><a href="#Joining-with-ATC-level-1-prescriptions-features" data-toc-modified-id="Joining-with-ATC-level-1-prescriptions-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Joining with ATC level 1 prescriptions features</a></span></li><li><span><a href="#Merging-split-rows" data-toc-modified-id="Merging-split-rows-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merging split rows</a></span><ul class="toc-item"><li><span><a href="#Invalid-discharges" data-toc-modified-id="Invalid-discharges-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Invalid discharges</a></span></li><li><span><a href="#Merging-short-transfers" data-toc-modified-id="Merging-short-transfers-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Merging short transfers</a></span></li><li><span><a href="#Invalid-transfers" data-toc-modified-id="Invalid-transfers-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Invalid transfers</a></span></li></ul></li><li><span><a href="#Demographic-features" data-toc-modified-id="Demographic-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Demographic features</a></span><ul class="toc-item"><li><span><a href="#Patients-table" data-toc-modified-id="Patients-table-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Patients table</a></span></li><li><span><a href="#Admissions" data-toc-modified-id="Admissions-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Admissions</a></span></li></ul></li><li><span><a href="#Joining-with-ATC-level-2-prescriptions-features" data-toc-modified-id="Joining-with-ATC-level-2-prescriptions-features-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Joining with ATC level 2 prescriptions features</a></span></li></ul></div>

In this notebook, we clean and extract features for the [transfers](https://mimic.mit.edu/docs/iv/modules/hosp/transfers/) table and add features from prescriptions (features extracted in other notebook), [admissions](https://mimic.mit.edu/docs/iv/modules/hosp/admissions/) and [patients](https://mimic.mit.edu/docs/iv/modules/hosp/patients/) tables. All data wrangling, cleaning and feature extraction is going to be done with one goal in mind: Predicting the hospital department/care unit a patient is going to end up next

In [25]:
import numpy as np
import pandas as pd
import copy
import nachopy
import importlib
import pickle
pd.options.mode.chained_assignment = None 

## Initial data exploration and cleaning

We will begin by cleaning and the data and performing a preliminary exploratory analysis (EDA on other notebook). 

In [15]:
transfers = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/transfers.csv')

If we sort values by subject_id and intime we can look at all the transfers for a patient in chronological order

In [16]:
transfers.sort_values(by = ['subject_id','intime']).head(10)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,NaN,2180-05-07 17:21:27,NaN
5,10000032,22841357.0,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
3,10000032,22841357.0,34100253,discharge,NaN,2180-06-27 18:49:12,NaN
9,10000032,29079034.0,32952584,ED,Emergency Department,2180-07-22 16:24:00,2180-07-23 05:54:00
13,10000032,29079034.0,39399961,ED,Emergency Department,2180-07-23 05:54:00,2180-07-23 14:00:00
14,10000032,29079034.0,39553978,admit,Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47
10,10000032,29079034.0,35888873,transfer,Transplant,2180-07-23 23:50:47,2180-07-24 19:52:58


So each row describes an event, that can be admission, ED (admission to Emergency Department?), discharge and transfer (among other possibly). In the table above, the first three rows show the first stay of patient 100000032. They arrived in the ED, where they spent 6 h, they were then transferred to the transplants care unti and were discharged after a day. As we want to predict the destination care unit for each transfer we should probably change the organisation of this df by adding the original care unit of the next transfer as a column of the previous row. This column, which we will call "event" will be our class (the outcome we want to predict). This way we can delete all discharge rows which are redundant anyway. 

Also note that some transfers appear to be continuous (rows with indices 9 and 13) but are two separate rows for some reason. We should merge these rows at some point as these are probably mistakes (a transfer to the same care unit does not make much sense). Let's explore and clean the data a bit more first

In [17]:
transfers['hadm_id'] = transfers.hadm_id.astype('Int64')
transfers.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991704 entries, 0 to 1991703
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   subject_id   1991704 non-null  int64 
 1   hadm_id      1644130 non-null  Int64 
 2   transfer_id  1991704 non-null  int64 
 3   eventtype    1991704 non-null  object
 4   careunit     1537380 non-null  object
 5   intime       1991704 non-null  object
 6   outtime      1537383 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 108.3+ MB


We have some null values for the care unit, hadm_ids and outtimes. The hadm_id null values are problematic as these can be useful to merge information from other tables. The care unit and outtime null values are probably related to the discharge events (above we saw that discharge events have no care unit). Let's convert the datetime columns to datetime. It does not make much sense exploring these as they are as the dates are randomised to deidentify the data. The dates for each patient are consistent, though, so we can use them to extract duration for example or to sort events for a given patient. We can create this new feature now

In [34]:
import datetime
# Let's have these as datetime, which will be useful to compare later
transfers['intime'] = pd.to_datetime(transfers.intime, infer_datetime_format=True)
transfers['outtime'] = pd.to_datetime(transfers.outtime, infer_datetime_format=True)
transfers['duration'] = (transfers.outtime - transfers.intime)/datetime.timedelta(hours = 1)

In [19]:
transfers.nunique()

subject_id      315460
hadm_id         454324
transfer_id    1991704
eventtype            4
careunit            39
intime         1980736
outtime        1532471
dtype: int64

Transfer id has as many unique values as the length of the df, so we will probably set this column as index later. This will come in handy to merge more tables in the future after feature expansion. We have 315460 patients and 454324 stays (with some missing values). The number of stays is higher than the number of patients because a patient may have more than one stay in the hospital (a stay can be considered a series of consecutive transfers). Let's see if we have any duplicated columns

In [20]:
# Checking for duplicates without the transfer_id column ensures we find any duplicate rows that were assigned different transfer_ids 
transfers_dupl = transfers.drop(columns = 'transfer_id').duplicated()
transfers = transfers[~transfers_dupl]
print(f'We deleted {transfers_dupl.sum()} duplicate rows')

We deleted 8 duplicate rows


Now let's see why we have missing values in outtime. Are these all for discharge events?

In [21]:
print(transfers[transfers.outtime.isna()].nunique())

subject_id     190279
hadm_id        454321
transfer_id    454321
eventtype           1
careunit            0
intime         454236
outtime             0
dtype: int64


No care units and only one unique eventtype, which we have seen above to be Discharge. We also know that the number of missing care units was the same as the number of missing outtimes (minus three) so practially all the missing care units are due to discharge events as well. Let's reorganise the dataframe so that we include the outcome/event in the same row as the original careunit. We can sort by subject_id and intime (inplace this time) and add a shifted series of the care unit as the "event" column in the df.

In [22]:
transfers.dropna(axis = 0, subset = 'careunit', inplace = True) # Dropping the 3 rows with missing care units we had seen before

display(transfers.tail(1)) # Last row is a discharge
transfers.sort_values(by=['subject_id','intime'],inplace = True)
events = transfers.careunit.loc[1:].values # From second row, getting the destination careunit (class)
transfers = transfers.iloc[:-1] # Until second-to-last row (last row is a discharge)
transfers['event'] = events # Adding the shifted series as new column

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
1991702,19999987,23865745,36195440,admit,Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30


If we have done it correctly, there should be one more null value in the new event column than for the discharge events, as these care unit null values correspond with discharge events.

In [23]:
print(sum(transfers.event.isna()))
print(sum(transfers.eventtype == 'discharge'))

0
0


It's correct, so we can fill the NaNs in the event column as discharges and drop the (old) discharge columns

In [24]:
transfers['event'] = transfers.event.fillna('Discharge') # Filling the null values with 'Discharge' event
transfers.reset_index(inplace = True,drop = True) # Resetting the index so it corresponds with sorted subject_id
print(f'Now we have {transfers.outtime.isna().sum()} null values in outtime')

Now we have 0 null values in outtime


## Extracting past care units features

We are going to be merging transfer rows and deleting others based on several criteria for data cleaning, but before, we can extract one important feature: past transfers. We do this before cleaning the transfers table because we are going to be deleting transfers that we do not want to predict (for example transfer with short durations).

We are going to extract the number of times (frequency) a patient has been in a care unit as a feature as well as the total duration of those past stays. 

In [21]:
# Initialising feature extraction df
prev_transfers = transfers[['subject_id','transfer_id','careunit','duration']]

# Creating new dummy variables from care units
prev_transfers = pd.concat([prev_transfers.drop(columns ='careunit'),\
                            pd.get_dummies(prev_transfers.careunit,prefix = 'freq'),
                            pd.get_dummies(prev_transfers.careunit,prefix = 'dur')],axis = 1)

# Now they are identical sets of dummy variables encoding the column "careunit", but we will fill these with past care units

# We can set the index as the subject_id as we will do this patient by patient
prev_transfers.set_index('subject_id',inplace = True)
dur_cols = prev_transfers.columns[41:] # Columns indexes for duration features

# Duration features are now equal to the duration of the stay in the current transfer (for the origin care unit)
prev_transfers[dur_cols] = prev_transfers[dur_cols].multiply(prev_transfers.duration.values,axis = 0).
prev_transfers.drop(columns = 'duration',inplace = True) # We drop duration as we don't need it anymore

cols = prev_transfers.columns[1:] # Dummy columns (to be edited)
# Iterating over each subject
subjects = prev_transfers.index.unique()
for subject in subjects:
    # Getting all rows for same subject
    summing_df = prev_transfers.loc[subject,cols]
    # Iterating over rows from the last (most recent) to the first (oldest)
    summing_len = len(summing_df)
    for i in range(summing_len-1,-1,-1):
        # Summing durations and frequencies 
        summing_df.iloc[i] = summing_df.iloc[:i].sum(axis = 0)
    # Features overwritten after summation
    prev_transfers.loc[subject,cols] = summing_df  
    print(f'{subject}/19999987', end = '\r') # Just a counter to check progress (really long script)

print()
# Setting index as transfer_id (and dropping the subject_id index) and saving pickle
prev_transfers.set_index('transfer_id', drop = True, inplace = True)
prev_transfers.to_pickle('G:/My Drive/Capstone Data/pickles/prev_transfers.pkl')

19999987/19999987


## Joining with ATC level 1 prescriptions features

As we are going to be merging rows, we can merge with prescriptions beforehand. This way, during the merging, we can just add the prescriptions of the two rows. We are creating a new feature on prescriptions, pres_number. This is just a counter of all the prescriptions in a given transfer. We will also be setting any remaining null values (right now -1) as 0.

In [45]:
prescriptions = pd.read_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_v1.pkl')
prescriptions.drop(columns = ['drug_type','drug','formulary_drug_cd','gsn','ndc'],inplace = True)
prescriptions[prescriptions.A<0] = 0 # Setting all null values to 0


# Initialising prescription number feature as 1
prescriptions['pres_number'] = 1
cat = list(prescriptions.drop(columns = ['subject_id','hadm_id','starttime']).columns)
cat.append('pres_number') # appending to categorical columns list for later

To merge the prescriptions categories to transfers we will have to use subject_id (missing values on hadm_id) and then remove all the rows for prescriptions whose start time is not between the intime and the outtime of the transfer. This will be done in a loop by chunks to avoid running out of memory.

In [100]:
# Initialising variables for loop
# Getting only necessary columns for the row selection
dfl = transfers[['subject_id','intime','outtime','transfer_id']].set_index('subject_id').sort_index()
# Adding hadm_id because it will be useful to fill hadm_id column on transfers
dfr = prescriptions.drop(columns = 'hadm_id').set_index('subject_id').sort_index() 

n = 15
nums = copy.deepcopy(cat)
non_nums = ['subject_id','intime','outtime','starttime']

df_merged = pd.DataFrame(columns = nums)

len_r = len(dfr)
max_l = dfl.index.max()
max_r = dfr.index.max()

inc = round(len(dfr)/n)
ind_r = copy.copy(inc)
sub_1 = 0
sub_2 = 0

for i in range(n):
    
    ind_r += inc 
    if ind_r < len_r:
        sub_2 = dfr.iloc[ind_r].name # Getting last index in increment 
    else:
        sub_2 = max_l # Getting last subject on left (transfers) df when reaching the end of the df
    
    dfr_chunk = dfr.loc[sub_1:sub_2]
    dfl_chunk = dfl.loc[sub_1:sub_2]
    # Inner join as any rows without prescriptions are lost anyway (will be recovered later)
    df_merging = pd.merge(dfl_chunk,dfr_chunk, on = 'subject_id') 

    # Selecting only rows where the prescription is between starttime and outtime 
    valid_rows = (df_merging.starttime>=df_merging.intime) & (df_merging.starttime<df_merging.outtime)
    
    df_merging = df_merging[valid_rows].reset_index(drop = False) # Putting subject_id away from index and back in columns
    # Retrieving hadm_id from prescriptions to merge later
    df_merging = df_merging.groupby('transfer_id')[nums].sum() # We sum to frequency encode prescriptions
    # Building merged dataframe
    df_merged = pd.concat([df_merged, df_merging],axis = 0)
    
    sub_1 = sub_2 + 1
    
    print(f'Loop {i + 1}/{n}',end='\r')
print()

# Setting index as transfer id and left joining df_merged
transfers.set_index('transfer_id',inplace = True)
transfers = transfers.join(df_merged)

Loop 15/15


Now we will have floats in the categories, let's see which data type is appropriate

In [46]:
print(f'Maximum value in categorical variables is {transfers2[cat].max().max()}')

Maximum value in categorical variables is 1109


Given that the maximum is 1109 (surprisingly high), we can set the data type as uint16 (int16 would be enough but same memory as int16 and we do not have negative values). 

In [158]:
transfers[cat] = transfers[cat].fillna(0) # Filling all missing values assuming that these transfers did not receive prescriptions
transfers[cat] = transfers[cat].astype('uint16')
transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537372 entries, 0 to 1537371
Data columns (total 24 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   transfer_id  1537372 non-null  int64         
 1   subject_id   1537372 non-null  int64         
 2   hadm_id      1189806 non-null  Int64         
 3   eventtype    1537372 non-null  object        
 4   careunit     1537372 non-null  object        
 5   intime       1537372 non-null  datetime64[ns]
 6   outtime      1537372 non-null  datetime64[ns]
 7   event        1537372 non-null  object        
 8   A            1537372 non-null  uint16        
 9   B            1537372 non-null  uint16        
 10  C            1537372 non-null  uint16        
 11  D            1537372 non-null  uint16        
 12  G            1537372 non-null  uint16        
 13  H            1537372 non-null  uint16        
 14  J            1537372 non-null  uint16        
 15  L            15

In [7]:
transfers.to_pickle('G:/My Drive/Capstone-Data/pickles/dirty_transpres.pkl') # Checkpoint

## Merging split rows

Next step is merging transfers that are invalid (discharge followed immediately by another transfer), those that are the same event (event = careunit) or that are very close together. We are also merging short transfers, as these will be scheduled at least a few hours in advance. We can simplify our dataset and improve our model by merging these rows when possible. To be conservative, we can set this lower transfer duration threshold at 3 h.

### Invalid discharges

We will first merge any discharge rows followed by a close transfer. We can do that by comparing and substituting values of shifted dataframe and series, as shown below

In [59]:
transfers.sort_values(by = ['subject_id','intime'],inplace = True) # Just in case

# Creating a boolean series to later select rows where discharge was closely followed by another transfer for same patient
disch_events = transfers.event.str.contains('Discharge').values & \
               (transfers.subject_id.shift(-1) == transfers.subject_id).values & \
               (transfers.outtime > transfers.intime.shift(-1) - np.timedelta64(3,'h')).values

print(f'There are {sum(disch_events)} discharge rows followed by another row.')

# Storing original length to know how many rows we remove in total
original_len = len(transfers)
transfers_len = np.inf # At least one loop
i = 0

# While there is a decrease in the number of rows
while len(transfers) < transfers_len:
    transfers_len = len(transfers)    
    
    # Same bool as before
    disch_events = transfers.event.str.contains('Discharge').values & \
               (transfers.subject_id.shift(-1) == transfers.subject_id).values & \
               (transfers.outtime > transfers.intime.shift(-1) - np.timedelta64(3,'h')).values
    
    # Bool for the following rows to these events
    disch_next = np.roll(disch_events,1)
    
    transfers.loc[disch_events,'outtime'] = transfers.outtime[disch_next].values # Outtime equal to that of next row
    transfers.loc[disch_events,'event'] = transfers.event[disch_next].values # Event equal to that of next row
    transfers.loc[disch_events,cat] += transfers.loc[disch_next,cat].values # Cats added for both rows
    
    # Removing the second row of each merged row (info now on first)
    transfers = transfers[~disch_next]
    
    i +=1
    
print('Number of loops:',i)
print('Rows merged:', original_len - len(transfers))

There are 22932 discharge rows followed by another row.
Number of loops: 3
Rows merged: 23025


We have removed 30k rows. After merging we kept applying the algorithm until there were no more disch_events.

### Merging short transfers

Now we will update duration and merge any rows with duration shorter than 3h. As introduced above, predicting such transfer will be difficult and possibly pointless.

In [60]:
transfers['duration'] = (transfers.outtime - transfers.intime)/np.timedelta64(1,'h')

original_len = len(transfers)
transfers_len = np.inf # At least one loop

i = 0

while len(transfers) < transfers_len:
    transfers_len = len(transfers)    
    # We first create a new column called duration, which can serve as a new feature
    # We are interested in rows with duration under 3 hours...
    short_transfers = (transfers['duration'] < 3).values
    # And whose preceding row has the same subject_id
    short_transfers *= (transfers.subject_id.shift() == transfers.subject_id).values
    # And where the intime is close to the outtime of the preceding row (within 3 h)
    short_transfers *= (transfers.intime < transfers.outtime.shift() + np.timedelta64(3,'h')).values
    # We can edit the intime now
    short_prev = np.roll(short_transfers,-1)
    
    transfers.loc[short_transfers,'intime'] = transfers.intime[short_prev].values
    transfers.loc[short_transfers,'careunit'] = transfers.careunit[short_prev].values
    transfers.loc[short_transfers,'eventtype'] = transfers.eventtype[short_prev].values
    transfers.loc[short_transfers,cat] += transfers.loc[short_prev,cat].values
    
    transfers.reset_index(drop = True, inplace = True)
    transfers['duration'] = (transfers.outtime - transfers.intime)/np.timedelta64(1,'h')
    
    transfers = transfers[~short_prev]
    i += 1
    
    
print('Number of loops:',i)
print('Rows merged:', original_len - len(transfers))

Number of loops: 3
Rows merged: 16425


Now we can do the same but on following rows

In [61]:
transfers_len = np.inf # At least one loop
i = 0
original_len = len(transfers)

while len(transfers) < transfers_len:
    transfers_len = len(transfers)    
    # We are interested in rows with duration under 3 hours...
    short_transfers = (transfers['duration'] < 3).values
    # And whose preceding row has the same subject_id
    short_transfers *= (transfers.subject_id == transfers.subject_id.shift(-1)).values
    # And where the intime is close to the outtime of the preceding row (within 3 h)
    short_transfers *= (transfers.outtime > transfers.intime.shift(-1) - np.timedelta64(3,'h')).values
    # # We can edit the intime now
    
    short_next = np.roll(short_transfers,1)
    transfers.loc[short_transfers,'outtime'] = transfers.outtime[short_next].values
    transfers.loc[short_transfers,'event'] = transfers.event[short_next].values
    transfers.loc[short_transfers,cat] += transfers.loc[short_next,cat].values
    
    transfers = transfers[~short_next]
    transfers.reset_index(drop = True, inplace = True)

    transfers['duration'] = (transfers.outtime - transfers.intime)/np.timedelta64(1,'h')
    i += 1
print('Number of loops',i)
print('Rows merged:', original_len - len(transfers))

Number of loops 2
Rows merged: 36988


### Invalid transfers

Let's look at rows where the event (or destination careunit) is equal to the current careunit and there is a following transfer. Not all the rows here are actually rows that have been split, as sometimes the first row is followed by a row of a different patient. We will have to remove these rows as a transfer to the same careunit does not make much sense

In [167]:
# Bool dataframe of rows equal to the indexes of the first row of the split rows
split_rows = transfers[['subject_id','outtime','event']] == transfers[['subject_id','intime','careunit']]\
.shift(-1).rename(columns = {'intime':'outtime','careunit':'event'}) # Renaming so that the comparison works

split_rows = split_rows.all(axis = 1) # Getting bool series that is true only when the three colums of booldf are true
split_rows = split_rows & (transfers.event == transfers.careunit) # Filtering only those that have the same event as careunit
# Outtime and event for these rows are now equal to the previous transfer
transfers[['outtime','event']] = transfers[['outtime','event']].where(~split_rows,transfers[['outtime','event']].shift(-1))

# Discarding second row of each merged row
non_splits = ~split_rows.shift().fillna(False) # Nas formed during shift
transfers = transfers[non_splits]

Let's repeat to make sure that we have taken care of all the split rows (maybe some were split into 3 or more)

In [170]:
split_rows = transfers[['subject_id','outtime','event']] == transfers[['subject_id','intime','careunit']]\
.shift(-1).rename(columns = {'intime':'outtime','careunit':'event'})
split_rows = split_rows.all(axis = 1)
split_rows = split_rows & (transfers.event == transfers.careunit)
print(split_rows.any())

False


No more split rows. Now we can check which events and careunits are the same without being continuous transfers

In [171]:
non_transfers = transfers.careunit == transfers.event

display(transfers[non_transfers])
print(transfers[non_transfers].info())cd
print(transfers[non_transfers].describe(include = 'all',datetime_is_numeric=True))
print((transfers.outtime[non_transfers]-transfers.intime[non_transfers]).describe())

,transfer_id,subject_id,hadm_id,eventtype,careunit,intime,outtime,event,A,B,...,J,L,M,N,P,R,S,V,pres_number,duration
11,39159098,10000048,<NA>,ED,Emergency Department,2126-11-22 18:45:00,2126-11-23 02:07:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,7.366667
16,39891608,10000102,<NA>,ED,Emergency Department,2136-12-20 11:49:00,2136-12-20 13:56:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,2.116667
17,32522732,10000108,<NA>,ED,Emergency Department,2163-09-16 16:34:00,2163-09-16 18:13:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,1.650000
18,39513268,10000108,<NA>,ED,Emergency Department,2163-09-24 16:14:00,2163-09-24 21:02:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,4.800000
20,38081480,10000115,<NA>,ED,Emergency Department,2154-12-10 02:04:00,2154-12-10 05:59:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,3.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294046,35122662,19999659,<NA>,ED,Emergency Department,2182-01-07 20:42:00,2182-01-08 02:10:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,5.466667
1294048,38224473,19999750,<NA>,ED,Emergency Department,2144-03-22 14:27:00,2144-03-22 18:47:00,Emergency Department,0,0,...,0,0,1,0,0,0,0,0,1,4.333333
1294049,38616897,19999782,<NA>,ED,Emergency Department,2156-11-09 21:37:00,2156-11-10 03:12:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,5.583333
1294091,34751321,19999829,<NA>,ED,Emergency Department,2186-06-14 11:31:00,2186-06-14 14:56:00,Emergency Department,0,0,...,0,0,0,0,0,0,0,0,0,3.416667


<class 'pandas.core.frame.DataFrame'>
Int64Index: 313266 entries, 11 to 1294098
Data columns (total 24 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   transfer_id  313266 non-null  int64         
 1   subject_id   313266 non-null  int64         
 2   hadm_id      10727 non-null   Int64         
 3   eventtype    313266 non-null  object        
 4   careunit     313266 non-null  object        
 5   intime       313266 non-null  datetime64[ns]
 6   outtime      313266 non-null  datetime64[ns]
 7   event        313266 non-null  object        
 8   A            313266 non-null  uint16        
 9   B            313266 non-null  uint16        
 10  C            313266 non-null  uint16        
 11  D            313266 non-null  uint16        
 12  G            313266 non-null  uint16        
 13  H            313266 non-null  uint16        
 14  J            313266 non-null  uint16        
 15  L            313266 non-null  ui

It is not clear why some transfers are from one department to the other, but these are not events that we can predict. We will drop these rows.

In [172]:
transfers = transfers[~non_transfers]
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896572 entries, 0 to 1294101
Data columns (total 24 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   transfer_id  896572 non-null  int64         
 1   subject_id   896572 non-null  int64         
 2   hadm_id      852415 non-null  Int64         
 3   eventtype    896572 non-null  object        
 4   careunit     896572 non-null  object        
 5   intime       896572 non-null  datetime64[ns]
 6   outtime      896572 non-null  datetime64[ns]
 7   event        896572 non-null  object        
 8   A            896572 non-null  uint16        
 9   B            896572 non-null  uint16        
 10  C            896572 non-null  uint16        
 11  D            896572 non-null  uint16        
 12  G            896572 non-null  uint16        
 13  H            896572 non-null  uint16        
 14  J            896572 non-null  uint16        
 15  L            896572 non-null  uin

Finally, we can remove any transfers with duration equal to 0. There are some rows left with durations under> 0 but under 3 h, but we will leave these for now

In [174]:
transfers[~(transfers.duration==0)]
print(f'There are {len(transfers)} left from an original 1991704')

There are 896572 left from an original 1991704


## Demographic features

We have decreased the number of our rows by more than half in this cleaning process. Let's join this table with features from other tables to get some demographic patient info.

### Patients table

We can start with the patients table. It contains gender, age and dod as relevant features. We can merge these on subject_id. The anchor_year is the random year assigned to the patient and the anchor_year_group is the real year range.

In [3]:
patients = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/patients.csv')

In [176]:
patients['dod'] = pd.to_datetime(patients.dod, infer_datetime_format = True)

Basic eda:

In [177]:
print(patients.info(show_counts = True))
print('\n')
print(patients.dod.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315460 entries, 0 to 315459
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         315460 non-null  int64         
 1   gender             315460 non-null  object        
 2   anchor_age         315460 non-null  int64         
 3   anchor_year        315460 non-null  int64         
 4   anchor_year_group  315460 non-null  object        
 5   dod                30636 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 14.4+ MB
None


count                   30636
unique                  19705
top       2129-11-19 00:00:00
freq                        7
first     2104-12-24 00:00:00
last      2212-01-22 00:00:00
Name: dod, dtype: object


Okay, no null values (other that the patients that did not die) and nothing weird on date of death (dod). Let's look at info on gender and age.

In [179]:
nachopy.display_sbs(
                [patients.gender.value_counts(normalize = True)*100,\

                patients.loc[~patients.dod.isna(),'gender'].value_counts()/patients.gender.value_counts()*100,\

                pd.concat([patients.anchor_age[patients.gender == 'M'].describe().rename('Males'),\
                patients.anchor_age[patients.gender == 'F'].describe().rename('Females')], axis = 1)],\

                titles = ['Patient genders', '% that died', 'Male and female age distributions']
)

Patient genders 
 
 
   
 gender 
 
 
 
 
 F 
 52.906549 
 
 
 M 
 47.093451 
 
 
 

 
 % that died 
 
 
   
 gender 
 
 
 
 
 F 
 8.775367 
 
 
 M 
 10.763255 
 
 
 

 
 Male and female age distributions 
 
 
   
 Males 
 Females 
 
 
 
 
 count 
 148561.000000 
 166899.000000 
 
 
 mean 
 49.026588 
 48.080480 
 
 
 std 
 20.146493 
 21.515040 
 
 
 min 
 18.000000 
 18.000000 
 
 
 25% 
 30.000000 
 28.000000 
 
 
 50% 
 49.000000 
 46.000000 
 
 
 75% 
 65.000000 
 65.000000 
 
 
 max 
 91.000000 
 91.000000

Okay, good for now. We do not have any death registered in our transfers tables. This should be an event different from discharge, as the features for patients that expired and patients that were discharged from the hospital will probably be significantly different.

We will add the columns age, gender and dod to the transfers table

In [180]:
transfers = pd.merge(transfers,patients[['subject_id','gender','anchor_age','dod']], on = 'subject_id',how = 'left')
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896572 entries, 0 to 896571
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   transfer_id  896572 non-null  int64         
 1   subject_id   896572 non-null  int64         
 2   hadm_id      852415 non-null  Int64         
 3   eventtype    896572 non-null  object        
 4   careunit     896572 non-null  object        
 5   intime       896572 non-null  datetime64[ns]
 6   outtime      896572 non-null  datetime64[ns]
 7   event        896572 non-null  object        
 8   A            896572 non-null  uint16        
 9   B            896572 non-null  uint16        
 10  C            896572 non-null  uint16        
 11  D            896572 non-null  uint16        
 12  G            896572 non-null  uint16        
 13  H            896572 non-null  uint16        
 14  J            896572 non-null  uint16        
 15  L            896572 non-null  uint

Less than 1% of patients in the transfers column died rather than get discharged. We will change the class to "Deceased" for these rows. We can drop the dod column and check for any missing values in the new columns. 

### Admissions

Now let's check what we can add from admissions (private info not shown)

In [182]:
admissions = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/admissions.csv')
# display(admissions.head())
print(admissions.info(show_counts = True))
print(admissions.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454324 entries, 0 to 454323
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   subject_id            454324 non-null  int64 
 1   hadm_id               454324 non-null  int64 
 2   admittime             454324 non-null  object
 3   dischtime             454324 non-null  object
 4   deathtime             9087 non-null    object
 5   admission_type        454324 non-null  object
 6   admission_location    454324 non-null  object
 7   discharge_location    328519 non-null  object
 8   insurance             454324 non-null  object
 9   language              454324 non-null  object
 10  marital_status        444608 non-null  object
 11  race                  454324 non-null  object
 12  edregtime             315460 non-null  object
 13  edouttime             315460 non-null  object
 14  hospital_expire_flag  454324 non-null  int64 
dtypes: int64(3), obje

Okay, let's do a basic EDA of some variables that could be interesting

In [183]:
columns = ['admission_type','discharge_location','insurance','language','marital_status','race']
nachopy.display_sbs([admissions[col].value_counts() for col in columns],max_rows = 17, suffix = 'value_counts')

,admission_type
EW EMER.,157487
EU OBSERVATION,100133
OBSERVATION ADMIT,55210
URGENT,47127
SURGICAL SAME DAY ADMISSION,35994
DIRECT EMER.,20584
DIRECT OBSERVATION,19698
ELECTIVE,11089
AMBULATORY OBSERVATION,7002
,discharge_location


Admission type seems quite relevant. Discharge location also looks relevant and should probably be used to update the class ("event"), at least the discharges. Insurance relevant as well, as medicaid and medicare patients could have different outcomes than privately-insured. English and marital status will be added for now, although probably little relevance. Race may be relevant, could be added simply as a dummy column (1 if white, 0 otherwise). We will join now and adjust later during the EDA. 

Because we have some missing values on the admissions IDs (hadm_id), we will fill some of these features using subject_id (race, insurance and language)

In [ ]:
# Storing the hadm_id column as is (with NaNs)
hadm_nas = transfers.hadm_id.isna()
transfers['hadm_id'] transfers.hadm_id.fillna(0) # For the merge only
columns.append('hadm_id')

# Joining to transfers
transfers = pd.merge(transfers,admissions[columns], how = 'left', on = 'hadm_id')
transfers['hadm_id'] = transfers.hadm_id.where(~hadm_nas) # Setting missing values as NaNs again
transfers.rename(columns = {'hadm_id_nans':'hadm_id'},inplace = True)
transfers.info()

To fill the missing values, we can see if there are any subject_id that has a hadm_id on the admissions table but not on the transfers table. For race insurance and language there are no missing values on the admissions table, so we can try to retrieve them by updating using subject_id for the first occurence of these values

In [185]:
columns.append('subject_id')
missing_rows = admissions[columns].groupby('subject_id').first()

transfers.reset_index(drop = False)
transfers.set_index('subject_id',drop = False,inplace = True)
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896572 entries, 10000032 to 19999987
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   transfer_id         896572 non-null  int64         
 1   subject_id          896572 non-null  int64         
 2   eventtype           896572 non-null  object        
 3   careunit            896572 non-null  object        
 4   intime              896572 non-null  datetime64[ns]
 5   outtime             896572 non-null  datetime64[ns]
 6   event               896572 non-null  object        
 7   A                   896572 non-null  uint16        
 8   B                   896572 non-null  uint16        
 9   C                   896572 non-null  uint16        
 10  D                   896572 non-null  uint16        
 11  G                   896572 non-null  uint16        
 12  H                   896572 non-null  uint16        
 13  J                   

We have increased our number of hadm_ids and also the other new columns. We could search the database to fill the missing values for some of the demographic features, but the improvement would be minimal. Plus, some of these features are probably not very relevant.

In [186]:
edstays = pd.read_csv('G:/My Drive/Capstone-Data/MIMIC IV/ed/edstays.csv')
edstays.head().drop(columns = ['race','gender']) # We will not show sensitive information that can help reidentify patients

,subject_id,hadm_id,stay_id,intime,outtime,arrival_transport,disposition
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,AMBULANCE,ADMITTED
3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,AMBULANCE,HOME
4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,AMBULANCE,ADMITTED


The ED collects different data than the hospital, which is unfortunate. We will leave these features as they are for now.carry out analysis regardless. It appears we can add race (we can just group by subject_id and then update the column with the first occurence.

In [187]:
missing_rows = edstays[['hadm_id','race','subject_id']].groupby('subject_id').first()
transfers.update(missing_rows)
transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896572 entries, 10000032 to 19999987
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   transfer_id         896572 non-null  int64         
 1   subject_id          896572 non-null  int64         
 2   eventtype           896572 non-null  object        
 3   careunit            896572 non-null  object        
 4   intime              896572 non-null  datetime64[ns]
 5   outtime             896572 non-null  datetime64[ns]
 6   event               896572 non-null  object        
 7   A                   896572 non-null  uint16        
 8   B                   896572 non-null  uint16        
 9   C                   896572 non-null  uint16        
 10  D                   896572 non-null  uint16        
 11  G                   896572 non-null  uint16        
 12  H                   896572 non-null  uint16        
 13  J                   

We are done with all the merging so for now, we will leave transfers_id as index, as it is a unique identifier of the rows. We will also save the df in our pickle jar as version 1 of our dataset. We have carried out an EDA and feature engineering in another notebook

In [189]:
transfers.to_pickle('G:/My Drive/Capstone-Data/pickles/transfers_v1.pkl')

## Joining with ATC level 2 prescriptions features

During the model optimization and feature selection phase, we have decided to increase the granularity of the prescriptions to add new features. Let's make a dataframe adding the transfer IDs to the prescriptions table so that we con join the new categories to the split datasets on the model optimization page. 

In [16]:
transfers = pd.read_pickle('G:/My Drive/Capstone-Data/pickles/transfers_v1.pkl')
pres = pd.read_pickle('G:/My Drive/Capstone-Data/pickles/prescriptions_v2.pkl')
lvl2_cat = list(pres.columns[8:]) # Selecting categoricla columns for merging
pres[lvl2_cat] = pres[lvl2_cat].where(pres.A01>0,0) # Setting -1 values (null) as NaN

We can apply our previous prescriptions merging script again

In [17]:
# Initialising variables for loop
# Getting only necessary columns for the row selection
transfers = transfers[['subject_id','intime','outtime','transfer_id']].set_index('subject_id').sort_index()
# Adding hadm_id because it will be useful to fill hadm_id column on transfers
pres = pres.drop(columns =[ 'hadm_id','drug_type','drug','formulary_drug_cd','gsn','ndc']).set_index('subject_id').sort_index() 

n = 50

df_merged = pd.DataFrame(columns = lvl2_cat)

len_r = len(pres)
max_l = transfers.index.max()
max_r = pres.index.max()

inc = round(len(pres)/n)
ind_r = copy.copy(inc)
sub_1 = 0
sub_2 = 0

for i in range(n):
    
    ind_r += inc 
    if ind_r < len_r:
        sub_2 = pres.iloc[ind_r].name # Getting last index in increment 
    else:
        sub_2 = max_l # Getting last subject on left (transfers) df when reaching the end of the df
    
    pres_chunk = pres.loc[sub_1:sub_2]
    transfers_chunk = transfers.loc[sub_1:sub_2]
    # Inner join as any rows without prescriptions are lost anyway (will be recovered later)
    df_merging = pd.merge(transfers_chunk,pres_chunk, on = 'subject_id') 

    # Selecting only rows where the prescription is between starttime and outtime 
    valid_rows = (df_merging.starttime>=df_merging.intime) & (df_merging.starttime<df_merging.outtime)
    
    df_merging = df_merging[valid_rows].reset_index(drop = True)
    # Retrieving hadm_id from prescriptions to merge later
    df_merging = df_merging.groupby('transfer_id')[lvl2_cat].sum() # We sum to frequency encode prescriptions

    df_merged = pd.concat([df_merged, df_merging],axis = 0)
    
    sub_1 = sub_2 + 1
    
    print(f'Loop {i + 1}/{n}',end='\r')
print()

Loop 50/50


Let's check if there are any empty columns

In [28]:
valid_cols = df_merged.max()>0
print(f'There are {(~valid_cols).sum()} columns with 0 prescriptions, we can remove these')
df_merged = df_merged[df_merged.columns[valid_cols]]
df_merged.info()

There are 36 columns with 0 prescriptions, we can remove these
<class 'pandas.core.frame.DataFrame'>
Int64Index: 736824 entries, 30000417 to 39999906
Data columns (total 51 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A01     736824 non-null  object
 1   A02     736824 non-null  object
 2   A06     736824 non-null  object
 3   A07     736824 non-null  object
 4   A09     736824 non-null  object
 5   A11     736824 non-null  object
 6   A12     736824 non-null  object
 7   A16     736824 non-null  object
 8   B01     736824 non-null  object
 9   B02     736824 non-null  object
 10  B03     736824 non-null  object
 11  B05     736824 non-null  object
 12  C01     736824 non-null  object
 13  C03     736824 non-null  object
 14  C04     736824 non-null  object
 15  C05     736824 non-null  object
 16  C10     736824 non-null  object
 17  D01     736824 non-null  object
 18  D02     736824 non-null  object
 19  D03     736824 non-null  object
 20

There was, we could remove 36 columns out of the total 50. The data type changed to object, we can change it back to int. Let's see which type we could use

In [33]:
df_merged.max().max()

41

We could do int8 (max below 128). We will use uint8 instead because it takes the same space and we are not expecting any negative values. We will save a pickle to eat it later.

In [35]:
df_merged = df_merged.astype('uint8')
print(df_merged.info())
df_merged.to_pickle('G:/My Drive/Capstone-Data/pickles/trans_pres.pkl')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 736824 entries, 30000417 to 39999906
Data columns (total 51 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A01     736824 non-null  uint8
 1   A02     736824 non-null  uint8
 2   A06     736824 non-null  uint8
 3   A07     736824 non-null  uint8
 4   A09     736824 non-null  uint8
 5   A11     736824 non-null  uint8
 6   A12     736824 non-null  uint8
 7   A16     736824 non-null  uint8
 8   B01     736824 non-null  uint8
 9   B02     736824 non-null  uint8
 10  B03     736824 non-null  uint8
 11  B05     736824 non-null  uint8
 12  C01     736824 non-null  uint8
 13  C03     736824 non-null  uint8
 14  C04     736824 non-null  uint8
 15  C05     736824 non-null  uint8
 16  C10     736824 non-null  uint8
 17  D01     736824 non-null  uint8
 18  D02     736824 non-null  uint8
 19  D03     736824 non-null  uint8
 20  D04     736824 non-null  uint8
 21  D06     736824 non-null  uint8
 22  D07     736

We are down to 51 features and 41.5 MB, which should make every merging very easy. These data will be used directly on the model optimization notebook